In [1]:
'''Analysis and Extrapolation of Electric Sales'''
'''Darius Kavarana & Ziad Bandak'''

'Darius Kavarana & Ziad Bandak'

In [2]:
'''Introduction'''



'Introduction'

In [3]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import matplotlib.ticker as ticker

In [4]:
'''Data Collection – done'''

'Data Collection – done'

In [5]:
# Load the CSV file
file_path = 'Electric_Vehicle_Population_Size_History_By_County.csv'
ev_data_df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
ev_data_df

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,April 30 2022,Wahkiakum,WA,Truck,0,0,0,1815,1815,0.00
1,May 31 2017,Adams,WA,Truck,0,0,0,5079,5079,0.00
2,October 31 2019,Collin,TX,Passenger,2,1,3,142,145,2.07
3,January 31 2024,Franklin,WA,Truck,10,0,10,20616,20626,0.05
4,February 28 2018,Grays Harbor,WA,Truck,0,0,0,20109,20109,0.00
...,...,...,...,...,...,...,...,...,...,...
21202,March 31 2021,Carson City,NV,Passenger,1,0,1,12,13,7.69
21203,February 28 2023,Adams,WA,Passenger,21,11,32,13009,13041,0.25
21204,June 30 2020,Douglas,NE,Passenger,0,1,1,55,56,1.79
21205,February 29 2020,Chelan,WA,Passenger,216,78,294,54600,54894,0.54


In [6]:
'''Step 2: Data Processing'''

'Step 2: Data Processing'

In [7]:
#explain no missing data. We cleaned it by getting rid of location data and merging the same dates

In [8]:
#datetime format, missing vals, correct data types/entries, battery electric vehicle, ev, phev, and total. 

#convert string date to integer value
# ev_data_df['Date'] = pd.to_datetime(ev_data_df['Date']).dt.strftime('%Y%m%d').astype(int)

ev_data_df['Date'] = pd.to_datetime(ev_data_df['Date']).dt.date

#grouping the data columns for the cleaned dataset. 
grouped_df = ev_data_df.groupby('Date').agg({
    'Battery Electric Vehicles (BEVs)': 'sum',
    'Plug-In Hybrid Electric Vehicles (PHEVs)': 'sum',
    'Electric Vehicle (EV) Total': 'sum',
    'Non-Electric Vehicle Total': 'sum',
    'Total Vehicles': 'sum'
}).reset_index()
grouped_df

,Date,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles
0,2017-01-31,14456,7535,21991,5950145,5972136
1,2017-02-28,14920,7730,22650,5937132,5959782
2,2017-03-31,15311,7929,23240,5935443,5958683
3,2017-04-30,15794,8120,23914,5930385,5954299
4,2017-05-31,16190,8330,24520,5931989,5956509
...,...,...,...,...,...,...
82,2023-11-30,126893,35948,162841,5978248,6141089
83,2023-12-31,130181,36606,166787,5961384,6128171
84,2024-01-31,134330,37692,172022,5934658,6106680
85,2024-02-29,137436,38384,175820,5886880,6062700


In [9]:
# Plotting the original cleaned dataset before any analysis. 
ev_data_df = ev_data_df.groupby('Date')['Electric Vehicle (EV) Total'].sum().reset_index()

plt.figure(figsize=(10, 5))
plt.scatter(ev_data_df['Date'], ev_data_df['Electric Vehicle (EV) Total'], marker='o')
plt.title('Total Electric Vehicles in Washington State Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Electric Vehicles')
plt.grid(True)
plt.show()

In [10]:
ev_totals_df = grouped_df[['Date', 'Electric Vehicle (EV) Total']]
ev_totals_df


# # Group by Date and sum up the total electric vehicles
# ev_totals = ev_data_df.groupby('Date')['Electric Vehicle (EV) Total'].sum().reset_index()

# # Convert 'Date' to datetime and sort by it
# ev_totals['Date'] = pd.to_datetime(wa_ev_totals['Date'])
# ev_totals = wa_ev_totals.sort_values('Date')

# # Display the summarized data
# wa_ev_totals.head()

,Date,Electric Vehicle (EV) Total
0,2017-01-31,21991
1,2017-02-28,22650
2,2017-03-31,23240
3,2017-04-30,23914
4,2017-05-31,24520
...,...,...
82,2023-11-30,162841
83,2023-12-31,166787
84,2024-01-31,172022
85,2024-02-29,175820


In [11]:
'''Exploratory Analysis and Data Visualization'''

'Exploratory Analysis and Data Visualization'

In [12]:
# # Plotting the data
# plt.figure(figsize=(10, 5))
# plt.plot(ev_totals_df['Date'], ev_totals_df['Electric Vehicle (EV) Total'], marker='o')
# plt.title('Total Electric Vehicles in Washington State Over Time')
# plt.xlabel('Date')
# plt.ylabel('Number of Electric Vehicles')
# plt.grid(True)
# plt.show()


In [13]:
#Data Collection for national vehicle registration. 
url = 'https://afdc.energy.gov/vehicle-registration'  # Confirm the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Depending on the structure which we need to identify first
data_rows = soup.find_all('tr')
state_ev_data = []
for row in data_rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        state = cells[0].text.strip()
        ev_count_str = cells[1].text.strip().replace(',', '')
        if ev_count_str.isdigit():
            ev_count = int(ev_count_str)
            state_ev_data.append((state, ev_count))

ev_registrations = pd.DataFrame(state_ev_data, columns=['State', 'EV Registrations'])
print(ev_registrations)


                   State  EV Registrations
0                Alabama              8700
1                 Alaska              2000
2                Arizona             65800
3               Arkansas              5100
4             California            903600
5               Colorado             59900
6            Connecticut             22000
7               Delaware              5400
8   District of Columbia              5900
9                Florida            168000
10               Georgia             60100
11                Hawaii             19800
12                 Idaho              5900
13              Illinois             66900
14               Indiana             17700
15                  Iowa              6200
16                Kansas              7600
17              Kentucky              7600
18             Louisiana              5900
19                 Maine              5000
20              Maryland             46100
21         Massachusetts             49400
22         

In [14]:
'''Model: Analysis, Hypothesis Testing, and Machine Learning'''

'Model: Analysis, Hypothesis Testing, and Machine Learning'

In [15]:


# Assuming 'wa_ev_totals' is already loaded and cleaned
wa_ev_totals['Date'] = pd.to_datetime(wa_ev_totals['Date'])
wa_ev_totals['TimeIndex'] = (wa_ev_totals['Date'] - wa_ev_totals['Date'].min()).dt.days  # Convert dates to a numeric value for regression

# Define the prediction endpoint
latest_date = wa_ev_totals['Date'].max()
target_year = 2028
years_to_predict = target_year - latest_date.year

# Generate future dates through 2034
future_times = pd.date_range(start=latest_date, periods=365 * years_to_predict, freq='D')
future_time_index_df = pd.DataFrame({'TimeIndex': (future_times - wa_ev_totals['Date'].min()).days})  # Use DataFrame instead of np.array

# Model fitting with Polynomial Regression
degree = 2  # Degree of polynomial features
poly_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
poly_model.fit(wa_ev_totals[['TimeIndex']], wa_ev_totals['Electric Vehicle (EV) Total'])  # Fit the model

# Predict future EV registrations using DataFrame to maintain feature names
future_preds_poly = poly_model.predict(future_time_index_df)

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(wa_ev_totals['Date'], wa_ev_totals['Electric Vehicle (EV) Total'], label='Historical EV Registrations')
plt.plot(future_times, future_preds_poly, label='Predicted EV Registrations Through 2034')
plt.title('Extended Future EV Registrations Prediction for Washington State')
plt.xlabel('Year')
plt.ylabel('Number of EV Registrations')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'wa_ev_totals' is not defined

In [ ]:


# Assuming the setup of the poly_model and wa_ev_totals are correct and previously defined

# Set the number of days from the start for the prediction date
future_date = pd.to_datetime('2034-12-31')
days_since_start = (future_date - wa_ev_totals['Date'].min()).days
future_time_index = np.array([[days_since_start]])

# Use the trained model to predict
future_pred = poly_model.predict(future_time_index)

# Calculate trend factor as the ratio of future prediction to the latest known data point
washington_latest_ev_count = wa_ev_totals['Electric Vehicle (EV) Total'].iloc[-1]
trend_factor = future_pred[0] / washington_latest_ev_count

# Apply this trend factor to the nationwide data
ev_registrations['Predicted 2034 Registrations'] = ev_registrations['EV Registrations'] * trend_factor

# Set display format to avoid scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Display the DataFrame with all states
print(ev_registrations[['State', 'EV Registrations', 'Predicted 2034 Registrations']])


In [ ]:


# Ensure data is sorted for better visual comparison
ev_registrations_sorted = ev_registrations.sort_values(by='Predicted 2034 Registrations', ascending=True)

# Creating a side-by-side bar graph with larger size
fig, ax = plt.subplots(figsize=(20, 15))
ind = np.arange(len(ev_registrations_sorted))  # the x locations for the groups
width = 0.35  # the width of the bars

rects1 = ax.barh(ind - width/2, ev_registrations_sorted['EV Registrations'], width, label='Current EV Registrations')
rects2 = ax.barh(ind + width/2, ev_registrations_sorted['Predicted 2034 Registrations'], width, label='Predicted 2034 Registrations')

# Add some text for labels, title and axes ticks
ax.set_xlabel('Number of EV Registrations', fontsize=14)
ax.set_title('Current and Predicted EV Registrations by State in 2034', fontsize=16)
ax.set_yticks(ind)
ax.set_yticklabels(ev_registrations_sorted['State'], fontsize=12)

# Format x-axis to use a logarithmic scale
ax.set_xscale('log')
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))  # Format with commas, no decimals

# Set legend and remove bar labels for a cleaner look
ax.legend(fontsize=14)

plt.show()


In [ ]:
'''Interpretation: Insight & Policy Decision'''